# About

This notebook extracts random points from polygons in the 'polygons_form_naip_images' folder and adds the spectral features at the point extracted from the corresponding NAIP scene. The output is a dataframe of points with the following features:

- geometry: coordinates of point *p* in ??? CRS
- naip_id: 
- polygon_id: 
- iceplant: whether point corresponds to a confirmed iceplant location or a confirmed non-iceplant location (0 = non-iceplant, 1= iceplant)
- r, g, b, nir: Red, Green, Blue and NIR bands values of NAIP scene with naip_id at at cooridnates defined by geometry

The dataframe is then saved in the 'temp' folder as a csv file.


'polygons_form_naip_images'
The polygons in confirmed locations as polygons on all available NAIP images over four regions regularly spaced along the Santa Barbara County coast: Carpinteria State Beach, the University of California, Santa Barbara campus, Gaviota State Park, and Point Conception. The polygons outlining confirmed ice plant outgrows are based on field observations and digitized records of ice plant locations from GBIF [*] and Calflora [*].


**NOTEBOOK VARIABLES:**

- year y from which we will be accessing the CFO California canopy height raster. y must be 2016, 2018 or 2020, since CFO's canopy height data is only available for these years.

- file path to which the Santa Barbara County subset of the canopy height data will be saved.

**OUTPUT:**

- canopy height raster layer for Santa Barbara County in the year y

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import geopandas as gpd

import extracting_points_from_polygons as pp

# Specify AOI and years from which to sample points

In [1]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

# aois = ['campus_lagoon','carpinteria','gaviota','point_conception']
aoi = 'gaviota'

# years = array of years, can be any subset from [2012, 2014, 2016, 2018, 2020]
# except if aoi = 'point_conception' (no pts for 2016)
years = [2016]


# ***************************************************
# ***************************************************

In [ ]:
all_pts = []

for year in years:
    
    # open polygons
    fp = pp.path_to_polygons(aoi,year)
    polys = gpd.read_file(fp)
    
    polys_ice = polys.loc[polys.iceplant==1]
    polys_ice.reset_index(inplace=True, drop=True)
    pts_ice = pp.naip_sample_proportion_no_warnings(polys_ice, 
                                                    polys.naip_id[0], 
                                                    0.9)  # sample 90% of pts in polygon

    print('*** '+str(year)+' # ice plant pts sampled')
    print(pts_ice.shape[0])
    pts_ice.polygon_id.value_counts()

    
    polys_nonice = polys.loc[polys.iceplant==0]
    polys_nonice.reset_index(inplace=True, drop=True)
    pts_nonice = pp.naip_sample_sliding_no_warnings(polys_nonice, polys.naip_id[0], 
                                                    0.9, # sample 90% of pts in polygon, but at most 1500
                                                    1500)

    print('*** '+str(year)+' # non ice plant pts sampled')
    print(pts_nonice.shape[0])
    pts_nonice.polygon_id.value_counts()
    
    pts = pd.concat([pts_ice,pts_nonice])
    print('*** '+str(year)+' # proportions')
    pp.iceplant_proportions(pts.iceplant)
    
    all_pts.append(pts)
    
    print( '---------------------------------------')

# Save points

In [ ]:
for i in range(0,len(years)):
    fp = pp.path_to_points_csv(aoi,years[i])
    pts[i].to_csv(fp)
        
